# LIBRARY

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk import tokenize
import re
nltk.download('stopwords')
import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hkhiy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [13]:
df=pd.read_excel(r'C:\Users\hkhiy\Documents\NCKH_Airbnb\Data1.xlsx')
df.head()

,reviewer_name,reviewer_country,rental_date,comments,langugage
0,Igor,"Kraków, Ba Lan",2023-12-02 00:00:00,The best support I have ever witnessed. Answer...,en
1,Kevin,"Cannes, Pháp",2023-12-04 00:00:00,Everything is very ok.,en
2,Walid,"Cannes, Pháp",2023-12-09 00:00:00,"Great accommodation for some games, the owner ...",en
3,Maxence,Vương quốc Anh,2023-12-11 00:00:00,Thanks Nigel & Tra for friendly stay in Ho Chi...,en
4,Lisa,Ai/len,2023-12-10 00:00:00,Nigel and others were very dedicated and respo...,en


In [16]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

stopwords_list = set(stopwords.words('english'))
class CustomPreprocess():
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''

    def __init__(self):
        pass
        
    def preprocess_text(self,sen):
        if pd.isna(sen):
            return ""
        sen = str(sen).lower()

        # Remove html tags
        sentence = remove_tags(sen)

        # Remove punctuations and numbers
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        
        # Single character removal
        sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

        # Remove multiple spaces
        sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.
        
        # Remove Stopwords
        pattern = re.compile(r'\b(' + r'|'.join(stopwords_list) + r')\b\s*')
        sentence = pattern.sub('', sentence)
         # Remove lỗi font
        sentence =  re.sub("[^\x00-\x7F]", " ", sentence)
        
        return sentence





In [17]:
test=CustomPreprocess()
X = []
sentences= list(df['comments'])
for sen in sentences:
    X.append(test.preprocess_text(sen))

In [18]:
X[159750]


'great place money friendly supportive staff '

In [20]:
df['comments'] = X
df.count()

reviewer_name       533508
reviewer_country    533294
rental_date         533515
comments            533515
langugage           125716
dtype: int64

In [22]:
df = df.dropna(subset=['comments'])
# Lưu dataframe sau khi xóa
df.head()




,reviewer_name,reviewer_country,rental_date,comments,langugage
0,Igor,"Kraków, Ba Lan",2023-12-02 00:00:00,best support ever witnessed answer fast kind s...,en
1,Kevin,"Cannes, Pháp",2023-12-04 00:00:00,everything ok,en
2,Walid,"Cannes, Pháp",2023-12-09 00:00:00,great accommodation games owner responded quickly,en
3,Maxence,Vương quốc Anh,2023-12-11 00:00:00,thanks nigel tra friendly stay ho chi minh cit...,en
4,Lisa,Ai/len,2023-12-10 00:00:00,nigel others dedicated responded quickly stay ...,en


In [23]:
import pandas as pd
from openpyxl import Workbook

df.to_csv(r'C:\Users\hkhiy\Documents\NCKH_Airbnb\after_preprocess.csv', encoding='utf-8')

# Create a new Excel workbook and save the DataFrame to an Excel file
workbook = Workbook()
worksheet = workbook.active

# Provide the correct file path using a raw string
with open(r'C:\Users\hkhiy\Documents\NCKH_Airbnb\after_preprocess.csv', 'r', encoding='utf-8') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        worksheet.append(row)

workbook.save("Data3.xlsx")